In [1]:
import pandas as pd
import numpy as np
import pickle
from datetime import datetime, timedelta
from tqdm import tqdm
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import plotly.express as px
#pd.set_option('display.float_format', lambda x: '%.6f' % x)
#pd.set_option('display.max_rows', 110)
import warnings
warnings.filterwarnings("ignore")
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
#from IPython.core.interactiveshell import InteractiveShell
#
#InteractiveShell.ast_node_interactivity = "last_expr"

In [2]:
# 25 functions with data in test set (last 3 days)
# workload_ids = [1, 7, 10, 11, 21, 31, 37, 40, 46, 47, 49, 50, 51, 52, 55, 56, 65, 66, 67, 68, 69, 71, 72, 80, 81]

workload_ids = [1, 7, 21, 31, 37, 50, 55, 56, 69, 71]

In [3]:
for i in workload_ids:
    data = pd.read_csv(f'../../data/training_data/{i}.txt', delimiter=',')
    data.drop(columns=["Unnamed: 0"], inplace=True)

    data.columns.values
    train_features = [feature for feature in data if feature != 'invocations']

    X = data[train_features]
    y = data[['invocations']]

    test_size = 0.2
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=False)
    model = xgb.XGBClassifier()
    model.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = model.predict(X_test)

    score = f1_score(y_test, y_pred)
    print(f'Model: {i}, Unique preds: {np.unique(y_pred)}, F1-score: {score}')

    with open(f"xgb_model_{i}.pkl", "wb") as f:
        pickle.dump(model, f)

Model: 1, Unique preds: [0 1], F1-score: 0.9136951924084818
Model: 7, Unique preds: [0 1], F1-score: 0.7904218120656841
Model: 21, Unique preds: [0 1], F1-score: 0.5426751592356688
Model: 31, Unique preds: [0 1], F1-score: 0.9996283909327388
Model: 37, Unique preds: [0 1], F1-score: 0.999628666914222
Model: 50, Unique preds: [0 1], F1-score: 0.8325508607198748
Model: 55, Unique preds: [0 1], F1-score: 0.7589498806682577
Model: 56, Unique preds: [0 1], F1-score: 0.8544548976203652
Model: 69, Unique preds: [0 1], F1-score: 0.5435801312089972
Model: 71, Unique preds: [0 1], F1-score: 0.8494623655913979


In [4]:
test_data = pd.concat([X_test, y_test], axis=1)

predictions = model.predict(test_data[train_features])
test_data["prediction"] = predictions

from datetime import datetime

now = datetime.now()
for i in tqdm(range(10)):
    predictions = model.predict(test_data[train_features]) # 3.5 days of historical data + latest datapoint

later = datetime.now()
difference = (later - now).total_seconds()
difference

100%|██████████| 10/10 [00:00<00:00, 24.83it/s]


0.414227